In [9]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import manifold, preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier as gpc
import math
import copy
from tqdm import tqdm
from scipy.sparse import csr_matrix, csgraph
from sklearn.cluster import KMeans
from matplotlib.lines import Line2D
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from collections import Counter
plt.style.use("seaborn-v0_8")
# sns.set(color_codes=False)
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath}')

seed = 10

In [7]:
# load german credit dataset
# 1000 instances with 20 features
data_path = "../data/raw_data/german/german_redone.csv"
german_df = pd.read_csv(data_path)
print(german_df)

     Checking-account  Months  Credit-history  Purpose  Credit-amount   
0                   1       6               4        3           1169  \
1                   2      48               2        3           5951   
2                   4      12               4        6           2096   
3                   1      42               2        2           7882   
4                   1      24               3        0           4870   
..                ...     ...             ...      ...            ...   
995                 4      12               2        2           1736   
996                 1      30               2        1           3857   
997                 4      12               2        3            804   
998                 1      45               2        3           1845   
999                 2      45               4        1           4576   

     Savings-account  Present-employment-since  Insatllment-rate   
0                  5                         5         

## Path findings with FaceLift

In [61]:
X = german_df.iloc[:, :20]
y = german_df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X = scaler.transform(X)
X_train_ = scaler.transform(X_train)
X_test_ = scaler.transform(X_test)

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(4, 3), max_iter=10000, random_state=seed)
clf.fit(X_train_, y_train)

Y_test_pred = clf.predict(X_test_)
Y_train_pred = clf.predict(X_train_)
Y_all_pred = clf.predict_proba(X)
print("test score:", clf.score(X_test_, y_test))
print("overall score", clf.score(X, y))

# number of negative instances in test set

print("number of negative instances in the test set:", (Y_test_pred == 0).sum())
print("number of negative instances in the entire dataset:", (Y_test_pred == 0).sum() + (Y_train_pred == 0).sum())


test score: 0.74
overall score 0.796
number of negative instances in the test set: 50
number of negative instances in the entire dataset: 242


In [51]:
def calculate_weighted_distance(v0, v1, penalty_term = 2):
    # [x1, x2]
    x_diff = v0[0] - v1[0]
    y_diff_sign = v0[1] - v1[1]
    y_diff = y_diff_sign if y_diff_sign >=0 else y_diff_sign*(-penalty_term)
    diff_norm = np.linalg.norm([x_diff, y_diff])
    return diff_norm

def get_weights_kNN(
    X, 
    n_neighbours = 20,
    penalty_term = 2,
    weight_func = None
    ):
    n_samples, n_ftrs = X.shape
    
    k = np.zeros((n_samples, n_samples))
    W = copy.deepcopy(k)
    # X = X.to_numpy()

    for i in tqdm(range(n_samples)):
        v0 = X[i]
        for j in range(n_samples):
            v1 = X[j]
            # modify the distance function so that removing pixels incurring larger cost.
            # dist = calculate_weighted_distance(v1, v0, penalty_term=penalty_term)
            dist = np.linalg.norm(v0 - v1)
            k[i, j] = dist

            if dist != 0:
                # dist should be greater than r
                W[i,j] = weight_func(dist)
        
        t = np.argsort(k[i, :])[(n_neighbours+1):]
        mask = np.ix_(t)
        k[i, mask] = 0
        # W[i, mask] = 0

    return k

def get_weights_kde(
    X, 
    n_neighbours = 20,
    density_scorer = None,
    weight_func = None
    ):
    n_samples, n_ftrs = X.shape
    
    k = np.zeros((n_samples, n_samples))
    W = copy.deepcopy(k)
    # X = X.to_numpy()

    for i in tqdm(range(n_samples)):
        v0 = X[i]
        for j in range(n_samples):
            v1 = X[j]
            dist = np.linalg.norm(v0 - v1)
            k[i, j] = dist

            if dist != 0:
                # dist should be greater than r
                mid_point = (v0 + v1)/2
                mid_point_log_density = density_scorer(mid_point.reshape(1, -1))
                if mid_point_log_density<0:
                    W[i,j] = 0
                else:
                    W[i, j] = weight_func(mid_point_log_density)*dist
        
        t = np.argsort(k[i, :])[(n_neighbours+1):]
        mask = np.ix_(t)
        k[i, mask] = 0
        W[i, mask] = 0

    return W

In [18]:
def construct_graph(weight_matrix):
    graph = csr_matrix(weight_matrix)
    return graph

def find_shortest_path(graph, start_point_idx):
    dist_matrix, predecessors = csgraph.dijkstra(
        csgraph=graph, directed=True, indices=start_point_idx, return_predecessors=True
    )
    return dist_matrix, predecessors

def reconstruct_shortest_path(predecessors, start_point_idx, end_point_idx):
    """Get all the nodes along the path between the start point and the end point. 

    Args:
        predecessors (matrix of shape (1, n_nodes)): contain the previous node in the path.
        start_point_idx (int): the index of the start data point
        end_point_idx (int): the index of the end data point

    Returns:
        node_path (list): [start_point_idx, intermedium points index, end_point_idx]
    """
    if predecessors[end_point_idx] == start_point_idx:
        node_path = [end_point_idx]
    else:
        node_path = []
    intermedium_idx = end_point_idx
    while (predecessors[intermedium_idx] != start_point_idx):
        node_path.append(intermedium_idx)
        intermedium_idx = predecessors[intermedium_idx]
    if intermedium_idx != node_path[-1]:
        node_path.append(intermedium_idx)
    node_path.append(start_point_idx)
    
    return node_path[::-1]
 
def build_symmetric_matrix(kernel):
    for i in range(kernel.shape[0]):
        for j in range(i):
            if kernel[j, i] != 0:
                kernel[i, j] = kernel[j, i]
            else:
                kernel[j, i] = kernel[i, j]
    return kernel

def build_asymmetric_matrix(kernel, X, weight_func, penalty_term):
    n_samples = kernel.shape[0]
    X = X.to_numpy()
    for i in tqdm(range(n_samples)):
        for j in range(n_samples):
            if kernel[i,j] != 0:
                v0 = X[i]
                v1 = X[j]
                dist = calculate_weighted_distance(v0, v1, penalty_term=penalty_term)
                kernel[j, i] = dist
    return kernel

In [49]:
kd_estimator = KernelDensity(kernel="gaussian", metric="l2", bandwidth=0.077)
kd_estimator.fit(X)
density_scorer = kd_estimator.score_samples

In [86]:
n_neighbours = 3
penalty_term = 100
n_samples, n_features = X.shape

def get_volume_of_sphere(d):
    return math.pi**(d/2)/math.gamma(d/2 + 1)

volume_sphere = get_volume_of_sphere(n_features)
r = (n_neighbours / (n_samples * volume_sphere))
# print(r)    # 0.010

# Construct the global weighted graph.
# Kernel is asymmetric if using KNN to get weight, and OG only keeps the bottom left half of the matrix

weight_func=lambda x: -x*np.log(r/x)  # x**alpha
kernel = get_weights_kNN(
            X,
            penalty_term=penalty_term,
            n_neighbours=int(n_neighbours),
            weight_func=weight_func
        )

# kernel = get_weights_kde(X, 
#                          n_neighbours=n_neighbours,
#                          weight_func=lambda x: 10/x,
#                          density_scorer=density_scorer
# )
sym_kernel = build_symmetric_matrix(kernel)
# asym_kernel = build_asymmetric_matrix(kernel, X, weight_func, penalty_term)


100%|██████████| 1000/1000 [00:06<00:00, 150.00it/s]


In [88]:
print(kernel[0][76])
# X_array = X.to_numpy()
x1 = X[0]
x78 = X[76]
dist = np.linalg.norm(x1 - x78)
log_den = density_scorer(((x1+x78)/2).reshape(1, -1))
print((1/log_den)*dist)
print(log_den)
print(dist)

0.0
[-0.06064321]
[-33.51292353]
2.0323311185410073


In [89]:
def get_minimum_dist(dist_matrix):
    """get the shortest distance and its data index
    Args:
        dist_matrix (array): shape: 1 x n_nodes

    Returns:
        min_dist: minimum distance in the distance matrix
        min_dist_idx: index of the data point with the shortest dist
    """
    min_dist = np.min(np.ma.masked_where(dist_matrix==0, dist_matrix, copy=False)) 
    min_dist_idx = np.argmin(np.ma.masked_where(dist_matrix==0, dist_matrix, copy=False))
    return min_dist, min_dist_idx

def get_closest_cf_point(dist_matrix, predictions, y, target_class, class_labels, num_paths = 1, pred_threshold=0.9):
    assert num_paths > 0 and isinstance(num_paths, int), "only positive integers"
    end_point_idx = []
    path_count = 0
    for idx in np.argsort(np.ma.masked_where(dist_matrix==0, dist_matrix)):
        if (y[idx] == target_class and
        predictions[idx, class_labels.index(target_class)] >= pred_threshold): 
            end_point_idx.append(idx)
            if path_count >= num_paths-1:
                break
            else:
                path_count += 1
    return end_point_idx

In [192]:
# calculate the shared percentage
# make sure the alternative points are epsilon distance away from each other

# print(cf_solutions[start_point_idx])

def calculate_shared_stop_point(sp_graph, path, alter_cf):
    stop_point = path[0]
    for idx, node in enumerate(path):
        # if not the last node
        if idx < len(path)-1:
            dist_matrix, _ = find_shortest_path(sp_graph, start_point_idx=node)
            dist_matrix_next, _ = find_shortest_path(sp_graph, start_point_idx=path[idx+1])
            if dist_matrix[alter_cf] < dist_matrix_next[alter_cf]:
                stop_point = node
                break
        # if the last node
        else:
            stop_point = node
    return stop_point

def calculate_shared_perc(sp_graph, path, stop_point):
    dist_matrix, _ = find_shortest_path(sp_graph, start_point_idx=path[0])
    perc = dist_matrix[stop_point]/dist_matrix[path[-1]]
    return perc

def dissimilarity(list1, list2):
    counter1 = Counter(list1)
    counter2 = Counter(list2)
    common_count = sum((counter1 & counter2).values())
    total_count = len(list1) + len(list2)
    return total_count - 2 * common_count

def find_alternative_cfs(all_path_dict, base_id = 0, count=5):
    # find top-count paths that are dissimilar to the base_id path
    dissimilarities = {key: dissimilarity(all_path_dict[base_id], value) for key, value in all_path_dict.items() if key != base_id}
    top_dissimilar_lists = sorted(dissimilarities, key=dissimilarities.get, reverse=True)[:count]
    return top_dissimilar_lists

stop_point = calculate_shared_stop_point(sp_graph, [4, 501, 611, 705], 393)
shared_perc = calculate_shared_perc(sp_graph, [1, 691, 834, 995], stop_point)
# print(stop_point)

def calculate_opportunity_potential(solution_dict, path_id):
    alter_cfs = find_alternative_cfs(solution_dict, base_id=path_id)
    # print(alter_cfs)

    avg_shared_perc = []
    for alter_path in alter_cfs:
        alter_end_node = solution_dict[alter_path][-1]
        first_path = solution_dict[0]
        # print(alter_end_node)
        stop_point = calculate_shared_stop_point(sp_graph, first_path, alter_end_node)
        shared_perc = calculate_shared_perc(sp_graph, first_path, stop_point)
        avg_shared_perc.append(shared_perc)
    # print(np.mean(avg_shared_perc))
    return(np.mean(avg_shared_perc))


{0: [5, 829, 886], 1: [5, 685, 791], 2: [5, 685, 406], 3: [5, 685, 406, 787], 4: [5, 685, 406, 933], 5: [5, 685, 406, 716], 6: [5, 685, 406, 939], 7: [5, 829, 886, 830], 8: [5, 685, 791, 940], 9: [5, 685, 406, 65], 10: [5, 87, 684, 42], 11: [5, 685, 791, 327], 12: [5, 685, 406, 933, 160], 13: [5, 685, 406, 716, 19], 14: [5, 87, 796, 277], 15: [5, 685, 406, 933, 499], 16: [5, 685, 406, 933, 223], 17: [5, 829, 946, 737, 529], 18: [5, 829, 946, 737, 923], 19: [5, 829, 886, 830, 178], 20: [5, 829, 886, 830, 473], 21: [5, 829, 886, 830, 863], 22: [5, 685, 406, 933, 160, 415], 23: [5, 685, 406, 716, 19, 854], 24: [5, 685, 406, 933, 499, 738], 25: [5, 87, 796, 277, 94], 26: [5, 87, 796, 277, 0], 27: [5, 87, 796, 277, 845, 254], 28: [5, 87, 796, 277, 94, 712], 29: [5, 87, 796, 277, 94, 781], 30: [5, 87, 796, 277, 94, 61], 31: [5, 87, 796, 277, 845, 299, 686], 32: [5, 87, 796, 277, 845, 299, 119], 33: [5, 87, 796, 277, 845, 299, 506], 34: [5, 87, 796, 277, 94, 781, 183], 35: [5, 87, 796, 277, 9

In [ ]:
sp_graph = construct_graph(sym_kernel)

start_point_idx = 5
target_class = 1
pred_threshold = 0.9
class_labels = list(map(int, clf.classes_))
cf_solutions = {}
cf_solutions[start_point_idx] = {}

dist_matrix, predecessors = find_shortest_path(sp_graph, start_point_idx=start_point_idx)

end_point_indices = get_closest_cf_point(dist_matrix, Y_all_pred, y, target_class, class_labels, num_paths=50, pred_threshold=pred_threshold)
print(end_point_indices)

for order, end_point_idx in enumerate(end_point_indices):
    shortest_path = reconstruct_shortest_path(predecessors, start_point_idx=start_point_idx, end_point_idx=end_point_idx)
    for node_idx in shortest_path[:-1]:
        cf_solutions[start_point_idx][order] = shortest_path
print(cf_solutions)

[886, 791, 406, 787, 933, 716, 939, 830, 940, 65, 42, 327, 160, 19, 277, 499, 223, 529, 923, 178, 473, 863, 415, 854, 738, 94, 0, 254, 712, 781, 61, 686, 119, 506, 183, 742, 484, 292, 928, 894, 150, 994, 628, 162, 71, 851, 549, 798, 81, 785]
{5: {0: [5, 829, 886], 1: [5, 685, 791], 2: [5, 685, 406], 3: [5, 685, 406, 787], 4: [5, 685, 406, 933], 5: [5, 685, 406, 716], 6: [5, 685, 406, 939], 7: [5, 829, 886, 830], 8: [5, 685, 791, 940], 9: [5, 685, 406, 65], 10: [5, 87, 684, 42], 11: [5, 685, 791, 327], 12: [5, 685, 406, 933, 160], 13: [5, 685, 406, 716, 19], 14: [5, 87, 796, 277], 15: [5, 685, 406, 933, 499], 16: [5, 685, 406, 933, 223], 17: [5, 829, 946, 737, 529], 18: [5, 829, 946, 737, 923], 19: [5, 829, 886, 830, 178], 20: [5, 829, 886, 830, 473], 21: [5, 829, 886, 830, 863], 22: [5, 685, 406, 933, 160, 415], 23: [5, 685, 406, 716, 19, 854], 24: [5, 685, 406, 933, 499, 738], 25: [5, 87, 796, 277, 94], 26: [5, 87, 796, 277, 0], 27: [5, 87, 796, 277, 845, 254], 28: [5, 87, 796, 277, 9

In [204]:
# find all negative instances
y_pred_bin = clf.predict(X)
all_neg_indices = [index for index, value in enumerate(y_pred_bin) if value == 0]
print(all_neg_indices)

# for a single instance, find one that is 
def find_solutions(sp_graph, start_point_idx, num_paths):
    target_class = 1
    pred_threshold = 0.9
    class_labels = list(map(int, clf.classes_))
    cf_solutions = {}
    cf_solutions[start_point_idx] = {}

    dist_matrix, predecessors = find_shortest_path(sp_graph, start_point_idx=start_point_idx)

    end_point_indices = get_closest_cf_point(dist_matrix, Y_all_pred, y, target_class, class_labels, num_paths=num_paths, pred_threshold=pred_threshold)
    # print(end_point_indices)
    
    for order, end_point_idx in enumerate(end_point_indices):
        shortest_path = reconstruct_shortest_path(predecessors, start_point_idx=start_point_idx, end_point_idx=end_point_idx)
        for node_idx in shortest_path[:-1]:
            cf_solutions[start_point_idx][order] = shortest_path
    # print(cf_solutions)
    return cf_solutions[start_point_idx], dist_matrix

avg_oppo_pot = []
avg_dist = []
for instance_idx in all_neg_indices:
    solution, dist_matrix = find_solutions(sp_graph, instance_idx, num_paths=30)
    # solution_dict = cf_solutions[start_point_idx]
    oppo_pot = calculate_opportunity_potential(solution, path_id=0)
    avg_oppo_pot.append(oppo_pot)
    avg_dist.append(dist_matrix[solution[0][-1]])
print(np.mean(avg_oppo_pot))
print(np.std(avg_oppo_pot))
print(np.mean(avg_dist))
print(np.std(avg_dist))

[1, 4, 9, 10, 11, 13, 14, 15, 18, 25, 29, 30, 31, 35, 44, 54, 56, 59, 62, 63, 75, 76, 79, 89, 95, 101, 113, 116, 120, 125, 126, 131, 138, 141, 142, 143, 148, 158, 163, 166, 170, 174, 175, 176, 181, 182, 189, 191, 197, 201, 203, 212, 218, 221, 226, 227, 235, 242, 252, 257, 261, 265, 272, 273, 274, 285, 286, 287, 289, 295, 301, 307, 315, 320, 321, 323, 332, 333, 334, 337, 338, 341, 353, 355, 359, 364, 368, 375, 378, 381, 387, 392, 396, 405, 412, 414, 416, 429, 431, 438, 442, 446, 457, 458, 462, 465, 466, 471, 472, 475, 481, 491, 496, 500, 501, 503, 504, 521, 522, 525, 528, 530, 538, 540, 545, 548, 552, 556, 557, 558, 560, 561, 566, 568, 569, 573, 578, 580, 583, 585, 588, 596, 597, 600, 602, 605, 607, 610, 612, 613, 623, 624, 630, 631, 637, 639, 640, 646, 648, 649, 650, 656, 663, 666, 677, 678, 701, 707, 711, 714, 719, 720, 721, 722, 727, 728, 730, 736, 739, 740, 743, 747, 751, 755, 762, 766, 771, 783, 789, 805, 808, 809, 812, 813, 814, 818, 819, 822, 826, 827, 831, 832, 835, 840, 849, 85

In [206]:
avg_oppo_pot = []
avg_dist = []
for instance_idx in all_neg_indices:
    solution, dist_matrix = find_solutions(sp_graph, instance_idx, num_paths=30)
    max_oppo = 0
    max_oppo_dist = 0
    for i in range(10):
        oppo_pot = calculate_opportunity_potential(solution, path_id=i)
        if oppo_pot>max_oppo:
            max_oppo = oppo_pot
            max_oppo_dist = dist_matrix[solution[i][-1]]
    avg_oppo_pot.append(max_oppo)
    avg_dist.append(max_oppo_dist)

print(np.mean(avg_oppo_pot))
print(np.std(avg_oppo_pot))
print(np.mean(avg_dist))
print(np.std(avg_dist))

0.6517980489426524
0.306225505908013
2.5398832673036282
0.9854628105825265
